首先，导入一些包

In [1]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
import numpy

In [6]:
import csv
with open('./dataset/boneage-training-dataset.csv') as csv_file:
    reader = csv.DictReader(csv_file)
len(reader)

TypeError: object of type 'DictReader' has no len()

In [3]:
data_dir = './dataset/'

先从读取数据集开始吧。首先，定义一个函数来读取CSV文件中的标签，并返回一个字典

In [4]:
def read_csv_traning(fname):
    # 读取fname来给标签字典返回一个文件夹名
    with open(fname, 'r') as f:
        # 第一列之所以不取得原因应该是序号吧
        lines = f.readlines()[1:]
    # rstrip([char]) 删除字符串末尾的指定字符，默认删除空格,并以','为分隔符
    tokens = [l.rstrip().split(',') for l in lines]
    # dict((key, value))
    # return dict(((id, name, value) for id, name, value in tokens))
    return tokens

# 测试一下
# labels = read_csv_traning(os.path.join(data_dir, 'boneage-training-dataset.csv'))
tokens = read_csv_traning(os.path.join(data_dir, 'boneage-training-dataset.csv'))
# dict(((ID, BoneAge, Male) for ID, BoneAge, Male in tokens))
tokens
# print('# 训练样本: ', len(tokens))


[['1377', '180', 'FALSE'],
 ['1378', '12', 'FALSE'],
 ['1379', '94', 'FALSE'],
 ['1380', '120', 'TRUE'],
 ['1381', '82', 'FALSE'],
 ['1382', '138', 'TRUE'],
 ['1383', '150', 'TRUE'],
 ['1384', '156', 'TRUE'],
 ['1385', '36', 'TRUE'],
 ['1387', '138', 'TRUE'],
 ['1388', '126', 'FALSE'],
 ['1389', '138', 'TRUE'],
 ['1390', '156', 'TRUE'],
 ['1391', '180', 'TRUE'],
 ['1393', '132', 'FALSE'],
 ['1394', '57', 'TRUE'],
 ['1395', '120', 'FALSE'],
 ['1396', '188', 'FALSE'],
 ['1398', '4', 'FALSE'],
 ['1399', '36', 'FALSE'],
 ['1400', '132', 'FALSE'],
 ['1402', '24', 'FALSE'],
 ['1403', '60', 'FALSE'],
 ['1404', '159', 'FALSE'],
 ['1405', '120', 'FALSE'],
 ['1406', '106', 'FALSE'],
 ['1407', '30', 'FALSE'],
 ['1408', '126', 'TRUE'],
 ['1409', '149', 'TRUE'],
 ['1411', '33', 'FALSE'],
 ['1412', '156', 'FALSE'],
 ['1414', '78', 'TRUE'],
 ['1415', '88', 'FALSE'],
 ['1416', '162', 'FALSE'],
 ['1417', '162', 'TRUE'],
 ['1418', '24', 'FALSE'],
 ['1419', '126', 'TRUE'],
 ['1420', '88', 'TRUE'],
 ['142

整理数据集
我们需要正路数据集来训练和测试模型。首先，定义reorg_train_valid函数来将验证集从原始的训练集中拆分出来。

In [5]:
labels = pd.read_csv(os.path.join(data_dir, 'boneage-training-dataset.csv'))


In [6]:
def copyfile(filename, target_dir):
    # 将文件复制到目标目录
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """将验证集从原始的训练集中拆分出来"""
    # 训练数据集中样本最少的类别中的样本数
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # 验证集中每个类别的样本数
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label


In [7]:
def reorg_test(data_dir):
    """在预测期间整理测试集，以方便读取"""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))

In [8]:
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = pd.read_csv(os.path.join(data_dir, 'boneage-training-dataset.csv'))
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [9]:
batch_size = 32 
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)

TypeError: 'numpy.ndarray' object is not callable